In [18]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks
import logging
import time
import sys

PROJECT_PATH = "/Users/shawn/Documents/personal/rsi_divergence_detector"
sys.path.append(PROJECT_PATH)
# Load the training data
# Replace 'training_data.csv' with your actual data file or DataFrame
df_total = pd.read_pickle(f'{PROJECT_PATH}/data/training_data.pickle')
divergence_df = pd.read_pickle(f"{PROJECT_PATH}/data/divergence_data2.pickle")

In [19]:
# Test dataframes sliced
df = df_total[df_total.timeframe == '15m']
df = df.loc[df.index >= '2024-11-20 00:00:00']
df = df.loc[df.index <= '2024-11-27 00:00:00']
# df = df.loc[df.index >= '2024-11-22 12:00:00']
# df = df.loc[df.index <= '2024-11-22 16:15:00']

df_div = divergence_df['15m']
df_div = df_div.loc[df_div.start_datetime >= '2024-11-20 00:00:00']
df_div = df_div.loc[df_div.index <= '2024-11-27 00:00:00']
df_div


AttributeError: 'DataFrame' object has no attribute 'start_datetime'

In [3]:
def find_previous_peak(df, divergence_start_idx, is_bullish, bullish_peak_rsi_threshold=55, bearish_peak_rsi_threshold=45):
    # Consider only data before the divergence start index
    divergence_start_idx
    df_before_divergence = df.loc[:divergence_start_idx]
    if is_bullish:
        # Bullish: Find previous peaks with RSI >= bullish_peak_rsi_threshold
        peaks_idx, _ = find_peaks(df_before_divergence['high'].values)
        if len(peaks_idx) == 0:
            return None
        valid_peaks = df_before_divergence.iloc[peaks_idx]
        valid_peaks = valid_peaks[valid_peaks['rsi'] >= bullish_peak_rsi_threshold]
        if valid_peaks.empty:
            return None
        return valid_peaks.index[-1]  # Return the index of the last valid peak
    else:
        # Bearish: Find previous valleys with RSI <= bearish_peak_rsi_threshold
        valleys_idx, _ = find_peaks(-df_before_divergence['low'].values)
        if len(valleys_idx) == 0:
            return None
        valid_valleys = df_before_divergence.iloc[valleys_idx]
        valid_valleys = valid_valleys[valid_valleys['rsi'] <= bearish_peak_rsi_threshold]
        if valid_valleys.empty:
            return None
        return valid_valleys.index[-1]  # Return the index of the last valid valley


def calculate_tp_sl(previous_idx, divergence_idx, df, is_bullish):
    previous_high = df.loc[previous_idx, 'high']
    divergence_low = df.loc[divergence_idx, 'low']
    previous_low = df.loc[previous_idx, 'low']
    divergence_high = df.loc[divergence_idx, 'high']

    if is_bullish:
        # For bullish: TP is at 0.382 Fibonacci level
        tp = divergence_low + (previous_high - divergence_low) * 0.382
        sl = divergence_low  # SL is the low at the divergence point
    else:
        # For bearish: TP is at 0.618 Fibonacci level
        tp = divergence_high - (divergence_high - previous_low) * (1 - 0.618)
        sl = divergence_high  # SL is the high at the divergence point
    return tp, sl

rsi_period=14     
min_bars_lookback=5     
max_bars_lookback=180                             
bullish_rsi_threshold=30     
bearish_rsi_threshold=70     
price_prominence=1     
rsi_prominence=1

In [4]:
price_high = df['high']
price_low = df['low']
rsi = df['rsi']
divergences = []

# Find peaks and troughs in price and RSI
price_high_peaks, _ = find_peaks(price_high, prominence=price_prominence)
price_low_peaks, _ = find_peaks(price_low, prominence=price_prominence)

rsi_peaks, _ = find_peaks(rsi.values, prominence=rsi_prominence)
rsi_troughs, _ = find_peaks(-rsi.values, prominence=rsi_prominence)

# Convert indices to arrays for vectorized operations
price_high_peaks_df_idx = df.index[price_high_peaks]
price_low_peaks_df_idx = df.index[price_low_peaks]
rsi_troughs_df_idx = set(df.index[rsi_troughs])
rsi_peaks_df_idx = set(df.index[rsi_peaks])

## Bullish Divergence

## Analyze

In [84]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd

# 서비스 계정 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(f'{PROJECT_PATH}/calcium-ember-444319-n7-3b60cf57e696.json', scope)
client = gspread.authorize(credentials)
# Google Service Account Credentials JSON 파일 경로
spreadsheet = client.open_by_url("https://docs.google.com/spreadsheets/d/1uJy2-CV63Pywc2GJJGRP6fSHXmPuTTybS8bMIWhH4Qc/edit?gid=0#gid=0")


In [ ]:
divergence_df2 = divergence_df
sheets = {}

for timeframe, ddf in divergence_df.items():
    ddf['TP_percent'] = 100 * (ddf['TP'] - ddf['entry_price']) * np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1) / ddf['entry_price'] 
    ddf['SL_percent'] = 100 * (ddf['entry_price'] - ddf['SL']) * np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1) / ddf['entry_price']
    ddf['TP_/_SL'] = ddf['TP_percent'] / ddf['SL_percent']
    is_bullish = np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1)
    ddf['profit'] = np.where(
        ddf['label'],
        is_bullish * (ddf['TP'] - ddf['entry_price']),
        -is_bullish * (ddf['entry_price'] - ddf['SL'])
    )
    if 'future_return' in ddf.columns:
        ddf = ddf.drop(columns=['future_return'])
    ddf[['price_change', 'rsi_change', 'TP', 'SL', 'TP_percent', 'SL_percent', 'TP_/_SL', 'profit']] = ddf[['price_change', 'rsi_change', 'TP', 'SL', 'TP_percent', 'SL_percent', 'TP_/_SL', 'profit']].round(2)
    ddf2 = ddf.copy()
    ddf2['start_datetime'] = ddf2.index
    # start_datetime 열을 가장 왼쪽으로 이동
    ddf2 = ddf2[['start_datetime'] + [col for col in ddf2.columns if col != 'start_datetime']]

    ddf2 = ddf2.astype(str)

    divergence_df2[timeframe] = ddf
    sheets[timeframe] = ddf2

pd.to_pickle(divergence_df2, f"{PROJECT_PATH}/data/divergence_data3.pickle")

for sheet_name, df in sheets.items():
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows="100", cols="20")
    worksheet.clear()  # 기존 데이터 삭제
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    
# TODO: in first sheet, add total profits

/var/folders/ns/shg9ht0545sbbr41sk4q_zgr0000gn/T/ipykernel_93962/804550579.py:35: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


In [ ]:
from collections import Counter

for timeframe in ['5m', '15m', '1h', '4h', '1d']:
    print(f"==========Analyzing {timeframe} profit and counts of position==========")
    ddf = divergence_df[timeframe]
    c = Counter(ddf.label)
    profit = 0
    for _, row in ddf.iterrows():
        is_bullish = 1 if row.divergence == "Bullish Divergence" else -1
        if row.label:
            profit += is_bullish * (row.TP - row.entry_price)
        else:
            profit -= is_bullish * (row.entry_price - row.SL)
    
    print(f"{timeframe} Total True / False count = {c}")
    print(f"{timeframe} Total profit = {profit}")
    print('\n')
    
    ddf_11 = ddf.loc[ddf.index >= '2024-11-01']
    c_11 = Counter(ddf_11.label)
    profit_11 = 0
    for _, row in ddf_11.iterrows():
        is_bullish = 1 if row.divergence == "Bullish Divergence" else -1
        if row.label:
            profit_11 += is_bullish * (row.TP - row.entry_price)
        else:
            profit_11 -= is_bullish * (row.entry_price - row.SL)
    print(f"{timeframe} after 24.11 True / False count = {c_11}")
    print(f"{timeframe} after 24.11 profit = {profit_11}")
    print('\n\n')



==========Analyzing 5m profit and counts of position==========
5m after 24.11 True / False count = Counter({False: 256, True: 95})
5m after 24.11 profit = 10430.97455999993



==========Analyzing 15m profit and counts of position==========
15m after 24.11 True / False count = Counter({False: 92, True: 41})
15m after 24.11 profit = 14577.046020000038



==========Analyzing 1h profit and counts of position==========
1h after 24.11 True / False count = Counter({False: 32, True: 7})
1h after 24.11 profit = -15634.050520000063



==========Analyzing 4h profit and counts of position==========
4h after 24.11 True / False count = Counter({False: 8, True: 2})
4h after 24.11 profit = -10419.922420000003



==========Analyzing 1d profit and counts of position==========
1d after 24.11 True / False count = Counter({False: 4})
1d after 24.11 profit = -8177.369999999995





In [73]:
ddf = divergence_df['15m']
ddf = ddf.loc[ddf.index >= '2024-11-01']
ddf['TP_percent'] = 100 * (ddf['TP'] - ddf['entry_price']) * np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1) / ddf['entry_price'] 
ddf['SL_percent'] = 100 * (ddf['entry_price'] - ddf['SL']) * np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1) / ddf['entry_price']
ddf['TP_vs_SL'] = ddf['TP_percent'] / ddf['SL_percent']
ddf['position'] = np.where(ddf['TP_percent'] >= 5*ddf['SL_percent'], True, False)
ddf.head(3)

/var/folders/ns/shg9ht0545sbbr41sk4q_zgr0000gn/T/ipykernel_93962/1241269880.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['TP_percent'] = 100 * (ddf['TP'] - ddf['entry_price']) * np.where(ddf['divergence'] == 'Bullish Divergence', 1, -1) / ddf['entry_price']
/var/folders/ns/shg9ht0545sbbr41sk4q_zgr0000gn/T/ipykernel_93962/1241269880.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['SL_percent'] = 100 * (ddf['entry_price'] - ddf['SL']) * np.where(ddf['divergence'] == 'Bullish Divergence', 1,

,end_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,future_return,TP,SL,label,TP_percent,SL_percent,position,TP_vs_SL
start_datetime,,,,,,,,,,,,,,,
2024-11-01 11:15:00,2024-11-01 12:45:00,2024-11-01 13:15:00,70202.83,2024-11-01 08:45:00,Bearish Divergence,997.14,12.204858,0.014173,70024.44978,70467.65,False,0.254093,0.377221,False,0.673590
2024-11-02 14:30:00,2024-11-03 01:00:00,2024-11-03 01:30:00,69118.68,2024-11-02 07:00:00,Bullish Divergence,-878.89,-18.948065,-0.012717,69353.57396,69100.01,False,0.339842,0.027012,True,12.581358
2024-11-03 02:30:00,2024-11-03 14:15:00,2024-11-03 14:45:00,67711.99,2024-11-02 16:45:00,Bullish Divergence,-17.94,4.004422,-0.000265,68414.60800,67672.00,False,1.037657,0.059059,True,17.569842


In [74]:
Counter(ddf.position)

Counter({False: 71, True: 62})

In [75]:
ddf = ddf.loc[ddf.position == True]
c_11 = Counter(ddf.label)
profit_11 = 0
for _, row in ddf.iterrows():
    is_bullish = 1 if row.divergence == "Bullish Divergence" else -1
    if row.label:
        profit_11 += is_bullish * (row.TP - row.entry_price)
    else:
        profit_11 -= is_bullish * (row.entry_price - row.SL)
print(f"{timeframe} after 24.11 True / False count = {c_11}")
print(f"{timeframe} after 24.11 profit = {profit_11}")
print('\n\n')

1d after 24.11 True / False count = Counter({False: 54, True: 8})
1d after 24.11 profit = 1615.2815400000982





In [55]:
ddf.loc[ddf.TP_vs_SL == -19482.617999574737]

,end_datetime,entry_datetime,entry_price,previous_peak_datetime,divergence,price_change,rsi_change,future_return,TP,SL,label,TP_percent,SL_percent,position,TP_vs_SL
start_datetime,,,,,,,,,,,,,,,
2017-11-25 13:30:00,2017-11-25 17:45:00,2017-11-25 18:15:00,8600.0,2017-11-25 03:15:00,Bearish Divergence,-49.64,-14.495652,-0.005776,8405.17382,8599.99,False,2.265421,-0.000116,True,-19482.618
